In [1]:
# 🛠️ 1. Setup
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import Counter

from torch.utils.data import Subset
import pickle

# 📦 Ensure data is downloaded and transform is applied
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# 📥 Load the EuroSAT dataset
dataset = datasets.EuroSAT(root="./data", transform=transform, download=True)

# Get class labels
class_names = dataset.classes
print("EuroSAT Classes:", class_names)


EuroSAT Classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [2]:
from torch.utils.data import DataLoader

In [3]:
def compute_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    mean = 0.0
    std = 0.0
    total_images = 0
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += batch_samples

    mean /= total_images
    std /= total_images
    return mean, std

In [4]:
mean, std = compute_mean_std(dataset)

In [5]:
mean

tensor([0.3444, 0.3803, 0.4078])

In [6]:
std

tensor([0.0914, 0.0651, 0.0552])

In [ ]:
from bayesian_torch.models.dnn_to_bnn import dnn_to_bnn

In [ ]:
# 📊 2. Basic Info
print(f"Total number of images: {len(dataset)}")
print(f"Number of classes: {len(class_names)}")

In [ ]:
# 📈 3. Count number of images per class
#labels = [dataset[i][1] for i in range(len(dataset))]
#label_counts = Counter(labels)

# Convert to readable format
#class_counts = {class_names[i]: label_counts[i] for i in range(len(class_names))}
#for cls, count in class_counts.items():
#    print(f"{cls:20s}: {count} images")

In [ ]:
# 📉 Optional: Visualize counts as a bar chart
#plt.figure(figsize=(10, 4))
#plt.bar(class_counts.keys(), class_counts.values())
#plt.title("Number of Images per Class in EuroSAT")
#plt.ylabel("Image Count")
#plt.xticks(rotation=45)
#plt.tight_layout()
#plt.show()

In [ ]:
# 🖼️ 4. Show Sample Images
def show_images(dataset, num_images=10):
    plt.figure(figsize=(15, 4))
    for i in range(num_images):
        img, label = dataset[i]
        img = img.permute(1, 2, 0)  # Convert [C,H,W] to [H,W,C]
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(class_names[label])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

show_images(dataset, num_images=10)


In [ ]:
# 🔍 5. Inspect Image Tensor
img_tensor, label = dataset[0]
print(f"Image tensor shape: {img_tensor.shape}")
print(f"Label index: {label} ({class_names[label]})")
print(f"Min/Max pixel value: {img_tensor.min():.2f}, {img_tensor.max():.2f}")


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, random_split

In [ ]:
# ----- 1. Bayesian Layer with Gaussian Weight Distribution -----
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features, prior_std=1.0):
        super().__init__()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.in_features = in_features
        self.out_features = out_features

        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-0.2, 0.2))
        self.weight_log_sigma = nn.Parameter(torch.Tensor(out_features, in_features).fill_(-5))
        self.bias_mu = nn.Parameter(torch.Tensor(out_features).uniform_(-0.2, 0.2))
        self.bias_log_sigma = nn.Parameter(torch.Tensor(out_features).fill_(-5))

        self.prior_std = prior_std

    def forward(self, x):
        weight_sigma = torch.exp(self.weight_log_sigma)
        bias_sigma = torch.exp(self.bias_log_sigma)

        # Sample weights
        weight_eps = torch.randn_like(weight_sigma)
        bias_eps = torch.randn_like(bias_sigma)
        weight = self.weight_mu + weight_sigma * weight_eps
        bias = self.bias_mu + bias_sigma * bias_eps

        self.kl = self._kl_divergence(weight, self.weight_mu, weight_sigma) + \
                  self._kl_divergence(bias, self.bias_mu, bias_sigma)

        return F.linear(x, weight, bias)

    def _kl_divergence(self, q_sample, mu, sigma):
        # KL divergence between posterior N(mu, sigma^2) and prior N(0, prior_std^2)
        prior_sigma = self.prior_std
        return torch.sum(
            torch.log(prior_sigma / sigma) +
            (sigma**2 + mu**2) / (2 * prior_sigma**2) - 0.5
        )

# ----- 2. Bayesian Neural Network -----
class BNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.b1 = BayesianLinear(64 * 64 * 3, 128)
        self.b2 = BayesianLinear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.b1(x))
        return self.b2(x)

    def kl_loss(self):
        return self.b1.kl + self.b2.kl

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = BNN()
print(f"Using device: {device}")
model = BNN().to(device)
model.load_state_dict(torch.load(r"results_eurosat/bnn_eurosat_20250612_171219.pth", map_location=torch.device(device)))
#model.load_state_dict(torch.load(r"results_eurosat/bnn_eurosat_20250612_173201.pth", map_location=torch.device(device)))
model.eval()

In [ ]:
image, label = dataset[3]
input_tensor = image.unsqueeze(0).to(device)  # Add batch dimension

# Forward pass with Bayesian inference
with torch.no_grad():
    outputs = model(input_tensor)
    probs = torch.softmax(outputs, dim=1)
    pred_class = probs.argmax().item()

true_tag = "XX"

if pred_class != class_names[label]:
    true_tag = "CORRECT!!!"

print(f"True label: {class_names[label]}")
print(f"Predicted: {class_names[pred_class]} (Confidence: {probs[0][pred_class]:.2f})")

#show the image with matplotlib, with the title as the predicted class with confidence and real class
plt.imshow(image.permute(1, 2, 0))
plt.title(true_tag + f" |||BNN\nPredicted: {class_names[pred_class]} (Confidence: {probs[0][pred_class]:.2f})\nTrue label: {class_names[label]}")

In [ ]:
# Define the same CNN model structure
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # -> (32, 32, 32)
        x = self.pool(F.relu(self.conv2(x)))   # -> (64, 16, 16)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Load the model
model_cnn = CNN()
model_cnn.load_state_dict(torch.load("results_eurosat/cnn_model.pth", map_location="cpu"))
model_cnn.eval()  # Set to evaluation mode

In [ ]:
transform_cnn = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3444, 0.3809, 0.4082], std=[0.1809, 0.1331, 0.1137])
])

# Load dataset
dataset_cnn_all = datasets.EuroSAT(root='./data', transform=transform_cnn, download=True)

with open('datasplit/split_indices.pkl', 'rb') as f:
    split = pickle.load(f)
    dataset_cnn_train = Subset(dataset, split['train'])
    dataset_cnn = Subset(dataset, split['test'])

In [ ]:
#image, label = dataset[25]
#input_tensor = image.unsqueeze(0).to(device)  # Add batch dimension
# Load a sample image
image, label = dataset_cnn[0]  # Pick any index
input_tensor = image.unsqueeze(0)  # Add batch dimension


# Forward pass with Bayesian inference
with torch.no_grad():
    output = model_cnn(input_tensor)  # Forward pass
    probabilities = F.softmax(output, dim=1)  # Confidence scores
    predicted_class = probabilities.argmax(dim=1).item()
    confidence = probabilities[0, predicted_class].item()

print(f"True label: {class_names[label]}")
print(f"Predicted: {class_names[predicted_class]} (Confidence: {probabilities[0][predicted_class]:.2f})")

true_tag = "XX"

if class_names[predicted_class] == class_names[label]:
    true_tag = "CORRECT!!!"

#show the image with matplotlib, with the title as the predicted class with confidence and real class
plt.imshow(image.permute(1, 2, 0).numpy() * 0.1137 + 0.4082)
plt.title(true_tag + f" ||| CNN\nPredicted: {class_names[predicted_class]} (Confidence: {probabilities[0][predicted_class]:.2f})\nTrue label: {class_names[label]}")

In [ ]:
class BCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # -> (32, 32, 32)
        x = self.pool(F.relu(self.conv2(x)))   # -> (64, 16, 16)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Load the model
model_bcnn = BCNN()

const_bnn_prior_parameters = {
    "prior_mu": 0.0,
    "prior_sigma": 1.0,
    "posterior_mu_init": 0.0,
    "posterior_rho_init": -3.0,
    "type": "Reparameterization",  # Flipout or Reparameterization
    "moped_enable": False,  # True to initialize mu/sigma from the pretrained dnn weights
    "moped_delta": 0.5,
    }

dnn_to_bnn(
    model_bcnn,
    const_bnn_prior_parameters
)

model_bcnn.load_state_dict(torch.load("results_eurosat/bayesian_cnn_model.pth", map_location="cpu"))
model_bcnn.eval()  # Set to evaluation mode

In [ ]:
image, label = dataset_cnn[0]  # Pick any index
input_tensor = image.unsqueeze(0)  # Add batch dimension


# Forward pass with Bayesian inference
with torch.no_grad():
    output = model_bcnn(input_tensor)  # Forward pass
    probabilities = F.softmax(output, dim=1)  # Confidence scores
    predicted_class = probabilities.argmax(dim=1).item()
    confidence = probabilities[0, predicted_class].item()

print(f"True label: {class_names[label]}")
print(f"Predicted: {class_names[predicted_class]} (Confidence: {probabilities[0][predicted_class]:.2f})")

true_tag = "XX"

if class_names[predicted_class] == class_names[label]:
    true_tag = "CORRECT!!!"

#show the image with matplotlib, with the title as the predicted class with confidence and real class
plt.imshow(image.permute(1, 2, 0).numpy() * 0.1137 + 0.4082)
plt.title(true_tag + f" ||| BCNN\nPredicted: {class_names[predicted_class]} (Confidence: {probabilities[0][predicted_class]:.2f})\nTrue label: {class_names[label]}")

In [ ]:
model_bcnn

In [ ]:
# make the plot as one, but show both bcnn and cnn predictions side by side
image, label = dataset_cnn[0]  # Pick any index
input_tensor = image.unsqueeze(0)  # Add batch dimension



plt.imshow(image.permute(1, 2, 0).numpy() * 0.1137 + 0.4082)

In [ ]:
dataset_cnn.indices

In [ ]:
len(dataset_cnn)

In [ ]:
len(dataset_cnn.indices)

In [ ]:
#image, label = dataset[25]
#input_tensor = image.unsqueeze(0).to(device)  # Add batch dimension
# Load a sample image

decision_threshold = 0.8  # Confidence threshold for known/unknown

dataset_cnn_indices = dataset_cnn.indices

image, label = dataset_cnn[0]  # Pick any index
input_tensor = image.unsqueeze(0)  # Add batch dimension


# Forward pass with Bayesian inference
with torch.no_grad():
    output = model_cnn(input_tensor)  # Forward pass
    probabilities = F.softmax(output, dim=1)  # Confidence scores
    predicted_class = probabilities.argmax(dim=1).item()
    confidence = probabilities[0, predicted_class].item()

#with torch.no_grad():
#    output_bcnn = model_bcnn(input_tensor)  # Forward pass
#    probabilities_bcnn = F.softmax(output_bcnn, dim=1)  # Confidence scores
#    predicted_class_bcnn = probabilities_bcnn.argmax(dim=1).item()
#    confidence_bcnn = probabilities_bcnn[0, predicted_class_bcnn].item()

outputs_bcnn = []
with torch.no_grad():
    for _ in range(20):  # 20 Monte Carlo samples
        output_bcnn = model_bcnn(input_tensor)
        prob_bcnn = F.softmax(output_bcnn, dim=1)
        outputs_bcnn.append(prob_bcnn)

#print the label for the highest probability for each outputs_bcnn entry
outputs_bcnn_label = [class_names[output_bcnn.argmax(dim=1)] for output_bcnn in outputs_bcnn]
print("BCNN Predictions (20 samples):", outputs_bcnn_label)

probabilities_bcnn = torch.stack(outputs_bcnn).mean(dim=0)  # shape: [1, num_classes]
predicted_class_bcnn = probabilities_bcnn.argmax(dim=1).item()

bcnn_confidences, bcnn_preds = probabilities_bcnn.max(dim=1)
print(bcnn_confidences)
print(bcnn_preds)

#print(f"True label: {class_names[label]}")
#print(f"Predicted: {class_names[predicted_class]} (Confidence: {probabilities[0][predicted_class]:.2f})")

#print(f"True label: {class_names[label]}")
#print(f"Predicted: {class_names[predicted_class]} (Confidence: {probabilities[0][predicted_class]:.2f})")

#true_tag = "XX"

#if class_names[predicted_class] == class_names[label]:
#    true_tag = "CORRECT!!!"

#show the image with matplotlib, with the title as the predicted class with confidence and real class
plt.imshow(image.permute(1, 2, 0).numpy() * 0.1137 + 0.4082)
plt.title(f"CNN Prediction: {class_names[predicted_class]} ({probabilities[0][predicted_class]:.2f})\n BCNN Prediction: {class_names[predicted_class_bcnn]} ({probabilities_bcnn[0][predicted_class_bcnn]:.2f})\nTrue label: {class_names[label]}")

In [ ]:
def ensemble_predict(model, x_batch, n_samples=20):
    """Returns mean prediction probabilities over n posterior samples"""
    preds = []
    model.eval()
    with torch.no_grad():
        for _ in range(n_samples):
            output = model(x_batch)  # stochastic forward pass
            prob = F.softmax(output, dim=1)
            preds.append(prob)
    
    mean_probs = torch.stack(preds).mean(dim=0)  # shape: [batch_size, num_classes]
    return mean_probs

In [ ]:
ensemble_predict(model_bcnn, input_tensor, n_samples=20)

In [ ]:
def predict_with_indecision(mean_probs, alpha=0.8):
    """Decide based on confidence threshold α"""
    confidences, pred_classes = mean_probs.max(dim=1)
    known_mask = confidences >= alpha
    unknown_mask = ~known_mask
    return mean_probs, pred_classes, confidences, known_mask, unknown_mask

In [ ]:
predict_with_indecision(ensemble_predict(model_bcnn, input_tensor, n_samples=20), alpha=0.8)

In [ ]:
def ensemble_predict_reproduce(model, x_batch, n_samples=20, seed=42):
    """Returns mean prediction probabilities over n posterior samples with reproducibility"""
    preds = []
    model.eval()
    
    # Set the seed for reproducibility
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    with torch.no_grad():
        for _ in range(n_samples):
            output = model(x_batch)  # stochastic forward pass
            prob = F.softmax(output, dim=1)
            preds.append(prob)
    
    mean_probs = torch.stack(preds).mean(dim=0)  # shape: [batch_size, num_classes]
    return mean_probs

In [ ]:
image, label = dataset_cnn[7]  # Pick any index
input_tensor = image.unsqueeze(0)  # Add batch dimension
reproduce_seed = 42  # Set a seed for reproducibility

### NORMAL CNN PREDICTION

with torch.no_grad():
    output = model_cnn(input_tensor)  # Forward pass
    probabilities = F.softmax(output, dim=1)  # Confidence scores
    predicted_class = probabilities.argmax(dim=1).item()
    confidence = probabilities[0, predicted_class].item()

### BAYESIAN CNN

decision_threshold = 0.8  # Confidence threshold for known/unknown

bcnn_mean_probs, bcnn_classes, bcnn_confidences, bcnnKnownMask, bcnnUnknownMask = predict_with_indecision(ensemble_predict_reproduce(model_bcnn, input_tensor, n_samples=20, seed=reproduce_seed), alpha=0.8)

bcnn_confidences, bcnn_preds = bcnn_mean_probs.max(dim=1)
print(bcnn_confidences)
print(bcnn_preds)

#show the image with matplotlib, with the title as the predicted class with confidence and real class
plt.imshow(image.permute(1, 2, 0).numpy() * 0.1137 + 0.4082)
plt.title(f"CNN Prediction: {class_names[predicted_class]} ({probabilities[0][predicted_class]:.2f})\n BCNN Prediction: {class_names[bcnn_preds]} ({float(bcnn_confidences):.2f})\nTrue label: {class_names[label]}")

In [ ]:
len(dataset_cnn)

In [ ]:
# create a loop for all images in dataset_cnn and record the prediction in a dataframe
import pandas as pd
from tqdm import tqdm

predictions_df = pd.DataFrame(columns=["Image Index", "CNN Predicted", "CNN Confidence", "BCNN Predicted", "BCNN Confidence", "True Label", "Reproduce Seed"])

total_len = len(dataset_cnn)
start_idx = int(total_len * 0)  # index where last 5% starts

for i in tqdm(range(start_idx, total_len)):

#for i in tqdm(range(len(dataset_cnn))):
#for i in range(5):
    image, label = dataset_cnn[i]  # Pick any index
    input_tensor = image.unsqueeze(0)  # Add batch dimension

    # Normal CNN prediction
    with torch.no_grad():
        output = model_cnn(input_tensor)  # Forward pass
        probabilities = F.softmax(output, dim=1)  # Confidence scores
        predicted_class = probabilities.argmax(dim=1).item()
        confidence = probabilities[0, predicted_class].item()

    # Bayesian CNN prediction
    bcnn_mean_probs, bcnn_classes, bcnn_confidences, bcnnKnownMask, bcnnUnknownMask = predict_with_indecision(
        ensemble_predict_reproduce(model_bcnn, input_tensor, n_samples=20, seed=reproduce_seed), alpha=0.8)

    bcnn_confidences, bcnn_preds = bcnn_mean_probs.max(dim=1)
    
    #print(f"Image {i}: CNN Predicted: {class_names[predicted_class]} ({probabilities[0][predicted_class]:.2f}), "
    #      f"BCNN Predicted: {class_names[bcnn_preds]} ({float(bcnn_confidences):.2f}), True label: {class_names[label]}")
    
    # Append to DataFrame
    new_row = pd.DataFrame([{
        "Image Index": i,
        "CNN Predicted": class_names[predicted_class],
        "CNN Confidence": probabilities[0][predicted_class].item(),
        "BCNN Predicted": class_names[bcnn_preds],
        "BCNN Confidence": float(bcnn_confidences),
        "BCNN Unknown": bcnnUnknownMask.item(),
        "True Label": class_names[label],
        "Reproduce Seed": reproduce_seed
    }])

    predictions_df = pd.concat([predictions_df, new_row], ignore_index=True)

# Save predictions to CSV with timestamp
# add timestamp to filename

import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
predictions_df.to_csv("predictions_eurosat_"+timestamp+".csv", index=False)

In [ ]:
len(dataset_cnn)

In [ ]:
import pandas as pd

In [ ]:
#predictions_df = pd.read_csv("predictions_eurosat_20250617_205043.csv")

In [ ]:
# compute the accuracy of the predictions
cnn_correct = (predictions_df["CNN Predicted"] == predictions_df["True Label"]).sum()
bcnn_correct = (predictions_df["BCNN Predicted"] == predictions_df["True Label"]).sum()
# compute accuracy for BCNN known classes
predictions_df["BCNN Known"] = predictions_df["BCNN Confidence"] >= 0.8
bcnn_known_correct = (predictions_df[predictions_df["BCNN Known"]]["BCNN Predicted"] == predictions_df[predictions_df["BCNN Known"]]["True Label"]).sum()
bcnn_known_total = predictions_df["BCNN Known"].sum()

print(f"CNN Accuracy: {cnn_correct / len(predictions_df) * 100:.2f}%")
print(f"BCNN Accuracy: {bcnn_correct / len(predictions_df) * 100:.2f}%")
print(f"BCNN Known Classes Accuracy: {bcnn_known_correct / bcnn_known_total * 100:.2f}%")

In [ ]:
# create a column, if the confidence is above 0.8, then it is a known class, otherwise unknown
predictions_df["BCNN Known"] = predictions_df["BCNN Confidence"] >= 0.8

In [ ]:
predictions_df.head()

In [ ]:
bcnn_known_total